## Shape class represents shapes that appear in the field

In [2]:
from enum import Enum
from utils import Point
import random

# Campo de Jogo

### Definição de tipo de célula do grid como definido nas especificações do jogo

In [3]:
class CellType(Enum):
	EMPTY = 0
	SHAPE = 1
	BLOCK = 2
	SOLID = 3
	
	
class Cell(object):
	
	def __init__(self, x=None, y=None, type=None):
		if x is not None and y is not None and type is not None:
			self.location = Point(x, y)
			self.state = type
			
		self.location = None
		self.state = CellType.EMPTY

	def is_out_of_boundaries(self, field):
		"""verify if piece is out of boundaries"""
		return self.location.X >= field.getWidth() or self.location.X < 0 or self.location.Y >= field.getHeight()
	
	def has_collision(self, field):
		"""verify if there is as collision os pieces"""
		cell = field.getCell(self.location.X, self.location.Y)
		return cell is not None and (self.state == CellType.SHAPE and (cell.is_solid() or cell.is_block()))
	
	def set_shape(self):
		"""set shape of new state"""
		self.state = CellType.SHAPE
	
	def set_location(self, x, y):
		"""set location of Point(x, y)"""
		if self.location is None:
			self.location = Point()
		
		self.location.set_location(x, y)
	
	def is_shape(self):
		"""verify if new state has same shape"""
		return self.state == CellType.SHAPE
	
	def is_solid(self):
		"""verify is state is solid"""
		return self.state == CellType.SOLID
	
	def is_block(self):
		"""verify is state is block"""
		return self.state == CellType.BLOCK
	
	def is_empty(self):
		"""verify is state is empty"""
		return self.state == CellType.EMPTY
	
	def get_state(self):
		"""returns the state of cell"""
		return self.state
	
	def get_location(self):
		"""return location of cell"""
		return self.location

### Definição de grid (campo de jogo) 

In [4]:
class Field(object):
	
	# Should be a 2D grid of Cell objects grid = [[Cell}}
	def __init__(self, width, height, input):
		self.width = width
		self.height = height
		self.grid = [[]]
		self.__parse_from_string(input)
	
	def __parse_from_string(self, input):
		self.grid = [[Cell() for j in range(self.width)] for i in range(self.height)]
		x, y = 0, 0
		for cellString in input.split(",|;"):
			cellCode = int(cellString.strip())
			self.grid[x][y] = Cell(x, y, CellType.values()[cellCode])
			
			if self.width == ++x:
				x = 0
				y += 1
				
	def get_cell(self, x, y):
		if x < 0 or x >= self.width or y < 0 or y >= self.height:
			return None
		
		return self.grid[x][y]
	
	def get_height(self):
		return self.height
	
	def get_width(self):
		return self.width


### Definição de formato das peças

In [5]:
class ShapeType(Enum):
    I = 'I'
    J = 'J'
    L = 'L'
    O = 'O'
    S = 'S'
    T = 'T'
    Z = 'Z'
    None


class Shape(object):
    """Shape class represents the shapes that appear in the field"""

    def __init__(self, type, field, location):
        self.type = type
        self.field = field
        self.blocks = [Cell for _ in range(4)]
        self.location = location
        self.size = 0
        self.shape = [[Cell for j in range(4)] for i in range(4)]

        self.__set_shape()
        self.set_block_locations()

    def __set_shape(self: object):
        if self.type == ShapeType.I:
            self.size = 4
            self.shape = initialize_shape()
            self.blocks[0] = self.shape[0][1]
            self.blocks[1] = self.shape[1][1]
            self.blocks[2] = self.shape[2][1]
            self.blocks[3] = self.shape[3][1]
        elif self.type == ShapeType.J:
            self.size = 3
            self.shape = initialize_shape()
            self.blocks[0] = self.shape[0][0]
            self.blocks[1] = self.shape[0][1]
            self.blocks[2] = self.shape[1][1]
            self.blocks[3] = self.shape[2][1]
        elif self.type == ShapeType.L:
            self.size = 3
            self.shape = initialize_shape()
            self.blocks[0] = self.shape[2][0]
            self.blocks[1] = self.shape[0][1]
            self.blocks[2] = self.shape[1][1]
            self.blocks[3] = self.shape[2][1]
        elif self.type == ShapeType.O:
            self.size = 2
            self.shape = initialize_shape()
            self.blocks[0] = self.shape[0][0]
            self.blocks[1] = self.shape[1][0]
            self.blocks[2] = self.shape[0][1]
            self.blocks[3] = self.shape[1][1]
        elif self.type == ShapeType.S:
            self.size = 3
            self.shape = initialize_shape()
            self.blocks[0] = self.shape[1][0]
            self.blocks[1] = self.shape[2][0]
            self.blocks[2] = self.shape[0][1]
            self.blocks[3] = self.shape[1][1]
        elif self.type == ShapeType.T:
            self.size = 3
            self.shape = initialize_shape()
            self.blocks[0] = self.shape[1][0]
            self.blocks[1] = self.shape[0][1]
            self.blocks[2] = self.shape[1][1]
            self.blocks[3] = self.shape[2][1]
        elif self.type == ShapeType.Z:
            self.size = 3
            self.shape = initialize_shape()
            self.blocks[0] = self.shape[0][0]
            self.blocks[1] = self.shape[1][0]
            self.blocks[2] = self.shape[1][1]
            self.blocks[3] = self.shape[2][1]

        for block in self.blocks:
            block.__set_shape()
            
    def initialize_shape(self):
        newshape = [[Cell for j in range(self.size)] for i in range(self.size)]
        
        for y in self.size:
            for x in self.size:
                newshape[x][y] = Cell()
        return newshape
    
    def turn_left(self):
        """rotates shape to the left counter-clockwise"""
        temp = self.transpose_shape()

        for y in self.size:
            for x in self.size:
                self.shape[x][y] = temp[x][self.size - y - 1]
        self.set_block_locations()
        
    def turn_right(self):
        """rotates shape to the right clockwise"""
        temp = self.transpose_shape()
        
        for x in self.size:
            self.shape[x] = temp[self.size - x - 1]
        
        self.set_block_lacations()
    
    def one_down(self):
        """moves shape one position down"""
        self.location.Y+=1
        self.set_block_locatations()
    
    def one_right(self):
        """moves shape one position to the right"""
        self.location.X+=1;
        self.set_block_locations()
    
    def one_left(self):
        """moves shape one position to the left"""
        self.location.X-=1;
        self.set_block_locations()
        
    def transpose_shape(self):
        """used for rotations. return transposed matrix os current shape"""
        temp = [[Cell() for j in self.size] for i in self.size]
        
        for y in self.size:
            for x in self.size:
                temp[y][x] = self.shape[x][y]
        return temp
        
    def set_block_rotations(self):
        """uses shape's current orientation and position to set the
        actual location os the block-type cells on the field"""
        for y in self.size:
            for x in self.size:
                if self.shape[x][y].is_shape():
                    self.shape[x][y].set_location(self.location.X + x, self.location.Y + y)
    
    def set_location(self, x, y):
        """sets point location"""
        self.location = Point(x, y)

    def get_blocks(self):
        return self.blocks

    def get_location(self):
        return self.location

    def set_type(self):
        return self.type

# Jogador

In [6]:
class Player(object):
    def __init__(self):
        self.field = None
        self.name = ""
        self.points = 0
        self.combo = 0
        self.skips = 0

# Movimentos

In [7]:
class MoveType(Enum):
    DOWN = "down"
    LEFT = "left"
    RIGHT = "right"
    TURNLEFT = "turnleft"
    TURNRIGHT = "turnright"
    DROP = "drop"

In [8]:
class Move(object):
    def __init__(self):
        self.moves = []
    
    def move(self, moves):
        self.moves = moves

# Bot

In [9]:
class BotState(object):
    def __init__(self):
        self.round_number = 0
        self.players = {} # using dictionary to represent a hash map {player: ''}
        self.timebank
        self.my_name
        self.current_shape
        self.next_shape
        self.shape_location
        self.MAX_TIMEBANK
        self.TIME_PER_MOVE
        self.FIELD_WIDTH
        self.FIELD_HEIGHT

class BotStarter(object):
    def __init__(self):
        self.random = random()
    
    def get_move(self, state):
        all_moves = []
        move = None
        moves = moves
        while move is not MoveType.DROP:
            #move = allMoves.get(this.random.nextInt(allMoves.size()));
            move = all_moves.pop(self.random.choice(self.all_moves))
            moves.append(move)
        return Move(moves)
        
        

--REMOVE AFTER--
Put main method here. From bot's specifications, there needs to have a main method to start bot
Probably we will need to transform all this notebook in just python files, that one we use to present the professor

In [10]:
class BotParser(object):
    def __init__(self, bot):
        self.bot = bot
        self.current_state = BotState()
    
    def run(self):
        while input(stdin):
            line = input().strip()
            if len(line) == 0:
                continue
            parts = line.split(" ")
            if parts[0] == "settings":
                parse_settings(parts[1], parts[2])
            elif parts[0] == "update":
                if parts[1] == "game":
                    parse_game_data(parts[2], parts[3])
                else:
                    parse_player_data(parts[1], parts[2], parts[3])
            elif parts[0] == "action":
                self.current_state.timebank = int(parts[2])
                move = self.bot.get_move(self.current_state)
                if move is not None:
                    print(move)
                else:
                    print('pass')
            else:
                print('unknown command')
    
    def parse_settings(self, key, value):
        try:
            if key == "timebank":
                time = int(value)
                self.current_state.MAX_TIMEBANK = time
                self.current_state.timebank = time
            elif key == "time_per_move":
                self.current_state.TIME_PER_MOVE = time
            elif key == "player_names":
                #review this
                for name in key:
                    player = Player(name)
                    self.current_state.players = {name: player}
            elif key == "your_bot":
                self.current_state.my_name = value
            elif key == "field_width":
                self.current_state.FIELD_WIDTH = int(value)
            elif key == "field_height":
                self.current_state.FIELD_HEIGHT = int(value)
            else:
                print("cannot parse settings input with key", key)
        except:
            print('cannot parse settings value', value," for key", key)
    
    def parse_game_data(self, key, value):
        try:
            if key == "round":
                self.current_state.round_number = int(value)
            elif key == "this_piece_type":
                self.current_state.current_shape = ShapeType.value #verify if this is really how it gets a value from an enum
            elif key == "next_piece_type":
                self.current_state.next_shape = ShapeType.value
            elif key == "this_piece_position":
                self.current_state.shape_location = ShapeType.value
            else:
                print("cannot parse game data input with key", key)
        except:
            print("cannot parse game data value", value,"for key", key)
    
    def parse_player_data(self, player_name, key, value):
        player = self.current_state.players = player_name
        
        if player is None:
            print("could not find player with name", player_name)
            exit(1)
            
        try:
            if key == "row_points":
                player.points = int(value)
            elif key == "combo":
                player.combo = int(value)
            elif key == "skips":
                player.skips = int(value)
            elif key == "field":
                field_width = self.current_state.FIELD_WIDTH
                field_height = self.current_state.FIELD_HEIGHT
                player.field = Field(field_width, field_height, value)
            else:
                print("cannot parse", palyer_name," data inout with key", key)
        except:
            print("cannot parse", player_name," data value", value,"for key", key)
    
    def parse_point(self, input):
        split = input.strip(",")
        return Point(int(split[0]), int(split[1]))